In [2]:
import json
#GETTING MY ID:
id = open('id.json', 'r')
mi = json.load(id)
my_id = mi["my_id"]

from azureml.core import Workspace

ws = Workspace.create(name="workspace",
                      subscription_id = my_id,
                      resource_group = "prueba2",
                      location = "centralus")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-5521844a82
Deploying StorageAccount with name workspacstoragef1df04960.
Deploying KeyVault with name workspackeyvault092d6ee5.
Deploying AppInsights with name workspacinsights3459968c.
Deployed AppInsights with name workspacinsights3459968c. Took 27.86 seconds.
Deploying Workspace with name workspace.
Deployed Workspace with name workspace. Took 21.7 seconds.
Registering model model


In [3]:
################ score.py ###################
scorepy = """
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('{mname}')
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)["data"]
        data = pd.DataFrame(data)

        result = model.predict(data)

        return json.dumps({"result": result.tolist()})
    
    except Exception as e:
        return json.dumps({"error": str(e)})

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(environment=virtual_env, entry_script="score.py")
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.4, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name="real-estates-service",
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [4]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn']) ## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\juane\AppData\Local\Temp\ipykernel_6132\944137596.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [5]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 105da2fa-702d-4836-b29a-77a42f6461c4
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 863d8fe2477848169a313b7e3a69f782.azurecr.io/azureml/azureml_6ade10047ac278b9562561a856ae1185 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for 

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 105da2fa-702d-4836-b29a-77a42f6461c4
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 863d8fe2477848169a313b7e3a69f782.azurecr.io/azureml/azureml_6ade10047ac278b9562561a856ae1185 locally. Please refer to